Autores Gabriel Conejo Valerio -2014093542 Nasser Brown Joseph Jimenez Zuñiga - 2016133677

In [21]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout 
from keras.optimizers import Adam  

Extraccion de Datos

In [ ]:
data_path = "SetDeDatos/UTKFace"  # Ruta al directorio de las imágenes
image_files = os.listdir(data_path)
images = []
etiquetas = []
for file in image_files:
    if file.endswith('.jpg'):
        # Carga la imagen en escala de grises
        image = cv2.imread(os.path.join(data_path, file), cv2.IMREAD_GRAYSCALE)
        # Extrae la edad del nombre del archivo (asumiendo que la edad es el primer número antes del guion bajo)
        edad = int(file.split('_')[0])
        images.append(image)
        etiquetas.append(edad)
# Convierte las listas en arreglos NumPy
X = np.array(images)
y = np.array(etiquetas)
# Divide el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

MLP (Perceptron Multicapa)

In [ ]:
class PerceptronMulticapa:
    def __init__(self, capas, F_activacion, alpha=0.1, epochs=1000):
        self.capas = capas
        self.alpha = alpha
        self.epochs = epochs 
        self.F_activacion = F_activacion
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            peso = np.random.randn(capas[i], capas[i + 1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i + 1])
            self.bias.append(bias)
    def activacion(self, x,act):
        if(act=="sigmoid"):
            return 1.0 / (1 + np.exp(-x))
        if(act=="relu"):
            return(np.maximum(0, x))
        if(act=="tanh"):
             return np.tanh(x)
    def activacion_derivada(self, x, act):
      if act == "sigmoid":
          return x * (1 - x)
      if act == "relu":
          return np.where(x <= 0, 0, 1)
      if act == "tanh":
          return 1 - np.tanh(x) ** 2
      return x
    def feedforward(self, X):
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x,self.F_activacion[i])
            capa_activacion.append(y)
        return capa_activacion
    def backpropagation(self, X, y, capa_activacion):
        error = capa_activacion[-1] - y
        deltas = [error]
        for i in reversed(range(1, len(self.capas) - 1)):
            delta = deltas[-1]
            activacion = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1]
            delta_anterior = np.dot(delta, self.pesos[i].T)
            deltas.append(delta_anterior * self.activacion_derivada(activacion,self.F_activacion[-1]))
        deltas = list(reversed(deltas))
        for i in range(len(self.capas) - 1):
            activacion_anterior = capa_activacion[i]
            delta = deltas[i]
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])
                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)
    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
def encode_labels(y):
    encoder = OneHotEncoder(sparse=False)
    y_encoded = encoder.fit_transform(y.reshape(-1, 1))
    return y_encoded

Feature Extractor

In [ ]:
data_path = "SetDeDatos/UTKFace"
image_files = os.listdir(data_path)
images = []
etiquetas = []
for file in image_files:
    if file.endswith('.jpg'):
        image = cv2.imread(os.path.join(data_path, file), cv2.IMREAD_GRAYSCALE)
        edad = int(file.split('_')[0])
        images.append(image)
        etiquetas.append(edad)
X = np.array(images)
y = np.array(etiquetas)
# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento_FE, X_prueba_FE, y_entrenamiento_FE, y_prueba_FE = train_test_split(X, y, test_size=0.2, random_state=42)
# Aplanar las imágenes para que sean bidimensionales
X_entrenamiento_FE = X_entrenamiento_FE.reshape(X_entrenamiento_FE.shape[0], -1)
X_prueba_FE = X_prueba_FE.reshape(X_prueba_FE.shape[0], -1)
# Definir el número de componentes principales deseado
n_componentes = 50
# Inicializar y ajustar el modelo PCA
pca = PCA(n_components=n_componentes)
X_entrenamiento_pca = pca.fit_transform(X_entrenamiento_FE)
X_prueba_pca = pca.transform(X_prueba_FE)

Red sin Feature Extractor

In [ ]:
perceptron_NoFE = PerceptronMulticapa(capas=[200, 150, 100, 103], F_activacion=["sigmoid", "relu", "tanh","sigmoid"], alpha=0.05, epochs=1000)
y_entrenamiento_encoded_NoFE = encode_labels(y_entrenamiento)
y_prueba_encoded_NoFE = encode_labels(y_prueba)
perceptron_NoFE.entrenar(X_entrenamiento, y_entrenamiento_encoded_NoFE, epochs=3000)
predicciones_encoded_NoFE = y_prueba_encoded_NoFE.predecir(X_prueba)
# Decodificar las predicciones (seleccionar la clase con probabilidad mas alta)
predicciones_NoFE = np.argmax(predicciones_encoded_NoFE, axis=1)
accuracy_NoFE = accuracy_score(y_prueba, predicciones_NoFE)
precision_NoFE = precision_score(y_prueba, predicciones_NoFE, average='weighted')
recall_NoFE = recall_score(y_prueba, predicciones_NoFE, average='weighted')
f1_NoFE = f1_score(y_prueba, predicciones_NoFE, average='weighted')
print("Metricas red especifica sin Feature Eng [200, 150, 100, 103]")
print(f"Accuracy: {accuracy_NoFE:.2f}")
print(f"Precision (weighted): {precision_NoFE:.2f}")
print(f"Recall (weighted): {recall_NoFE:.2f}")
print(f"F1-score (weighted): {f1_NoFE:.2f}")

Red con Feature Extractor

In [ ]:
perceptron_FE = PerceptronMulticapa(capas=[n_componentes, 150, 100, 103], F_activacion=["sigmoid", "relu", "tanh", "sigmoid"], alpha=0.05, epochs=3000)
perceptron_FE.entrenar(X_entrenamiento_pca, y_entrenamiento_FE, epochs=3000)
predicciones_encoded_FE = perceptron_FE.predecir(X_prueba_pca)
# Decodificar las predicciones (seleccionar la clase con probabilidad más alta)
predicciones_FE = np.argmax(predicciones_encoded_FE, axis=1)
accuracy_FE = accuracy_score(y_prueba_FE, predicciones_FE)
precision_FE = precision_score(y_prueba_FE, predicciones_FE, average='weighted')
recall_FE = recall_score(y_prueba_FE, predicciones_FE, average='weighted')
f1_FE = f1_score(y_prueba_FE, predicciones_FE, average='weighted')
print("Metricas red especifica con Feature Eng [200, 150, 100, 103]")
print(f"Accuracy: {accuracy_FE:.2f}")
print(f"Precision (weighted): {precision_FE:.2f}")
print(f"Recall (weighted): {recall_FE:.2f}")
print(f"F1-score (weighted): {f1_FE:.2f}")

Comparacion entre Red sin Feature Extractor vs Red con Feature Extractor

In [ ]:
print("Comparacion entre Red especifica [200, 150, 100, 103] con Feature Extractor y sin Feature Extractor")
print(f"Accuracy con Feature Extractor: {accuracy_NoFE:.2f}")
print(f"Precision con Feature Extractor: {precision_NoFE:.2f}")
print(f"Recall con Feature Extractor: {recall_NoFE:.2f}")
print(f"F1-score con Feature Extractor: {f1_NoFE:.2f}")
print("vs")
print(f"Accuracy sin Feature Extractor: {accuracy_FE:.2f}")
print(f"Precision sin Feature Extractor: {precision_FE:.2f}")
print(f"Recall sin Feature Extractor: {recall_FE:.2f}")
print(f"F1-score sin Feature Extractor: {f1_FE:.2f}")

Redes Convolucionales

Filtrado de imagenes offline

Filtro Bilateral

In [ ]:
def filtro_bilateral(imagen, d, sigma_color, sigma_space):
    imagen_filtrada = cv2.bilateralFilter(imagen, d, sigma_color, sigma_space)
    return imagen_filtrada

Red Convolucional 1 en imagenes crudas

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(-1, 28, 28, 1)
X_train_CCN1, X_test_CCN1, y_train_CCN1, y_test_CCN1 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
y_train_CCN1 = to_categorical(y_train_CCN1.astype(np.int), num_classes=10)
y_test_CCN1 = to_categorical(y_test_CCN1.astype(np.int), num_classes=10)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding="same",strides=(1,1) ,activation='relu',input_shape=(28, 28, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1))) # este imput_shape no es necesario, ya que se hace solo
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_CCN1, y_train_CCN1, batch_size=128, epochs=10, verbose=1)
loss_CCN1, accuracy_CCN1 = model.evaluate(X_test_CCN1, y_test_CCN1, verbose=0)
print("Red Convolucional 1 en imagenes crudas")
print(f"Pérdida en el conjunto de prueba: {loss_CCN1}")
print(f"Precisión en el conjunto de prueba: {accuracy_CCN1}")
print ("La arquitectura del modelo es:  ")
model.summary()

Red Convolucional 1 con "Bilateral Filter"

In [ ]:
data_path = "SetDeDatos/UTKFace" 
image_files = os.listdir(data_path)
images = []
etiquetas = []
for file in image_files:
    if file.endswith('.jpg'):
        # Carga la imagen en escala de grises
        image = cv2.imread(os.path.join(data_path, file), cv2.IMREAD_GRAYSCALE)
        # Aplicar el filtro bilateral a la imagen
        imagen_filtrada = filtro_bilateral(image, d=9, sigma_color=75, sigma_space=75)
        # Extraer la edad del nombre del archivo (asumiendo que la edad es el primer numero antes del guion bajo)
        edad = int(file.split('_')[0])
        images.append(imagen_filtrada)  # Usar la imagen filtrada en lugar de la original
        etiquetas.append(edad)
X = np.array(images)
y = np.array(etiquetas)
X_entrenamiento_CCN1_FB, X_prueba_CCN1_FB, y_entrenamiento_CCN1_FB, y_prueba_CCN1_FB = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_CCN1_FB = to_categorical(y_entrenamiento_CCN1_FB.astype(np.int), num_classes=10)
y_test_CCN1_FB = to_categorical(y_prueba_CCN1_FB.astype(np.int), num_classes=10)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding="same",strides=(1,1) ,activation='relu',input_shape=(28, 28, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1))) # este imput_shape no es necesario, ya que se hace solo
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(y_entrenamiento_CCN1_FB, y_train_CCN1_FB, batch_size=128, epochs=10, verbose=1)
loss_CCN1_FB, accuracy_CCN1_FB = model.evaluate(y_prueba_CCN1_FB, y_test_CCN1_FB, verbose=0)
print("Red Convolucional 2 con Filtro Bilateral")
print(f"Pérdida en el conjunto de prueba: {loss_CCN1_FB}")
print(f"Precisión en el conjunto de prueba: {accuracy_CCN1_FB}")
print ("La arquitectura del modelo es:  ")
model.summary()

Red Convolucional 2 en imagenes crudas

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(-1, 28, 28, 1)
X_train_CCN2, X_test_CCN2, y_train_CCN2, y_test_CCN2 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Convertir las etiquetas a one-hot encoding
y_train_CCN2 = to_categorical(y_train_CCN2.astype(np.int), num_classes=10)
y_test_CCN2 = to_categorical(y_test_CCN2.astype(np.int), num_classes=10)
model = Sequential()
# Agregar capas convolucionales y de pooling
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Agregar una segunda capa convolucional y de pooling
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Agregar una capa de aplanamiento
model.add(Flatten())
# Agregar una capa completamente conectada con dropout para regularización
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout con una tasa del 50%
model.add(Dense(10, activation='softmax'))
# Compilar el modelo con el optimizador Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X_train_CCN2, y_train_CCN2, batch_size=128, epochs=10, verbose=1)
loss_CCN2, accuracy_CCN2 = model.evaluate(X_test_CCN2, y_test_CCN2, verbose=0)
print("Red Convolucional 2 en imagenes crudas")
print(f"Pérdida en el conjunto de prueba: {loss_CCN2}")
print(f"Precisión en el conjunto de prueba: {accuracy_CCN2}")
print("La arquitectura del modelo es:")
model.summary()

Red Convolucional 2 con "Bilateral Filter"

In [ ]:
data_path = "SetDeDatos/UTKFace"  # Ruta al directorio de las imágenes
image_files = os.listdir(data_path)
images = []
etiquetas = []
for file in image_files:
    if file.endswith('.jpg'):
        # Carga la imagen en escala de grises
        image = cv2.imread(os.path.join(data_path, file), cv2.IMREAD_GRAYSCALE)
        # Aplicar el filtro bilateral a la imagen
        imagen_filtrada = filtro_bilateral(image, d=9, sigma_color=75, sigma_space=75)
        # Extraer la edad del nombre del archivo (asumiendo que la edad es el primer número antes del guion bajo)
        edad = int(file.split('_')[0])
        images.append(imagen_filtrada)  # Usar la imagen filtrada en lugar de la original
        etiquetas.append(edad)
X = np.array(images)
y = np.array(etiquetas)
X_entrenamiento_CCN2_FB, X_prueba_CCN2_FB, y_entrenamiento_CCN2_FB, y_prueba_CCN2_FB = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_CCN2_FB = to_categorical(y_train_CCN1_FB.astype(np.int), num_classes=10)
y_test_CCN2_FB = to_categorical(y_test_CCN1_FB.astype(np.int), num_classes=10)
model = Sequential()
# Agregar capas convolucionales y de pooling
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Agregar una segunda capa convolucional y de pooling
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Agregar una capa de aplanamiento
model.add(Flatten())
# Agregar una capa completamente conectada con dropout para regularización
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout con una tasa del 50%
model.add(Dense(10, activation='softmax'))
# Compilar el modelo con el optimizador Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X_entrenamiento_CCN2_FB, y_train_CCN2_FB, batch_size=128, epochs=10, verbose=1)
loss_CCN2_FB, accuracy_CCN2_FB = model.evaluate(X_prueba_CCN2_FB, y_test_CCN2_FB, verbose=0)
print("Red Convolucional 2 con Filtro Bilateral")
print(f"Pérdida en el conjunto de prueba: {loss_CCN2_FB}")
print(f"Precisión en el conjunto de prueba: {accuracy_CCN2_FB}")
print("La arquitectura del modelo es:")
model.summary()

Tabla de Metricas

Respuesta a las preguntas
¿Cual es el tamano de los modelos?
¿Cual es la arquitectura (cuales y cuantas capas se utilizaron)?
¿Que ventajas tiene utilizar un feature extractor? referirse a las metricas y tiempos
¿Que modelo se entrena mas rapido?
¿Cuales secciones de la imagen son las mas importantes para la eleccion?
¿Cual fue el mejor modelo?